In [17]:
from itertools import chain, combinations
import numpy as np  # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sys
import os
from sklearn.tree import DecisionTreeClassifier
# Trova il percorso assoluto della cartella "esotic" (dove si trova il tuo ipynb)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Trova il percorso assoluto della cartella "lib"
lib_dir = os.path.join(current_dir, "..", "lib")

# Aggiungi il percorso di "lib" al PYTHONPATH
sys.path.append(lib_dir)

from Utility import *

In [18]:
dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2', 'TrainRecall',
                                  'TrainROC', 'TestAccuracy', 'TestPrecision', 'TestF2', 'TestRecall', 'TestROC', "UnderTestAccuracy"
"OverTestAccuracy", 'BestParams'])

features = [
    'EloDiff', 
    "PointsDifference",
    "HomeGoalsRatio",
    "AwayGoalsRatio",
    "GoalsDifference",
    "ConcededGoalsDifference",
    'GoalsAvgSum',
    'RecentFormRatio',
    "EloRatio",
    "HomeLast3Points",
    "HomeAvgGoalsScored",
    "HomeAvgGoalsConceded",
    "HomeEwmaPoints",
    "HomeEwmaGoalsScored",
    "HomeEwmaGoalsConceded",
    "AwayLast3Points",
    "AwayAvgGoalsScored",
    "AwayAvgGoalsConceded",
    "AwayEwmaPoints",
    "AwayEwmaGoalsScored",
    "AwayEwmaGoalsConceded",
    "HomeWins",
    "HomeDraws",
    "HomeLosses",
    "AwayWins",
    "AwayDraws",
    "AwayLosses",
    "UltimoScontroDiretto",
    "Last3PointsDifference",
    "GoalRatioDifference",
    "EwmaGoalsSum",
    "GoalsSum",
    "HomePointGap",
    "AwayPointGap",
    "HomeGoalGap",
    "AwayGoalGap",
    "feat1",
    "feat2",
    'FormDiff',
    'RecentFormDiff', 
    'GoalCumulativeSum',
    # 'GoalCumulativeSumPrev'
]
classWeight={1: 1, 0: 2}
# features = ['AwayGoalsRatio', 'GoalsDifference', 'HomeEwmaGoalsScored']
combinazioni = chain.from_iterable(combinations(
    features, r) for r in range(1,3))#len(features) + 1))

combos = []

## Combo

In [19]:
for idx, combinazione in enumerate(combinazioni):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    print(idx, time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
    combos.append(combinazione)

print('Length of combos',combos.__len__())

0 22:12:36 : ('EloDiff',)
1 22:12:36 : ('PointsDifference',)
2 22:12:36 : ('HomeGoalsRatio',)
3 22:12:36 : ('AwayGoalsRatio',)
4 22:12:36 : ('GoalsDifference',)
5 22:12:36 : ('ConcededGoalsDifference',)
6 22:12:36 : ('GoalsAvgSum',)
7 22:12:36 : ('RecentFormRatio',)
8 22:12:36 : ('EloRatio',)
9 22:12:36 : ('HomeLast3Points',)
10 22:12:36 : ('HomeAvgGoalsScored',)
11 22:12:36 : ('HomeAvgGoalsConceded',)
12 22:12:36 : ('HomeEwmaPoints',)
13 22:12:36 : ('HomeEwmaGoalsScored',)
14 22:12:36 : ('HomeEwmaGoalsConceded',)
15 22:12:36 : ('AwayLast3Points',)
16 22:12:36 : ('AwayAvgGoalsScored',)
17 22:12:36 : ('AwayAvgGoalsConceded',)
18 22:12:36 : ('AwayEwmaPoints',)
19 22:12:36 : ('AwayEwmaGoalsScored',)
20 22:12:36 : ('AwayEwmaGoalsConceded',)
21 22:12:36 : ('HomeWins',)
22 22:12:36 : ('HomeDraws',)
23 22:12:36 : ('HomeLosses',)
24 22:12:36 : ('AwayWins',)
25 22:12:36 : ('AwayDraws',)
26 22:12:36 : ('AwayLosses',)
27 22:12:36 : ('UltimoScontroDiretto',)
28 22:12:36 : ('Last3PointsDifference',

In [20]:
for idx, combo in enumerate(combos):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    feat_cols = [col for col in combo]

    data = getFullData().copy()
    data = data.dropna(subset=feat_cols)
    data['isATS'] = np.where(((data['FTAG'] > 0)), 1, 0)

    # dataTrain = getTrainOver1Data().copy()
    # dataTest = getTestOver1Data().copy()
    # dataTrain = dataTrain.dropna(subset=feat_cols)
    # dataTest = dataTest.dropna(subset=feat_cols)
    # print(len(dataTrain), len(dataTest))


    # x_train = dataTrain[feat_cols]
    # y_train = dataTrain['isOver']
    # x_test = dataTest[feat_cols]
    # y_test = dataTest['isOver']
    x_train, x_test, y_train, y_test = train_test_split(
        data[feat_cols], data['isATS'], test_size=0.3, random_state=42, shuffle=True, stratify=data['isATS'])

    x_train = x_train.sort_index()
    x_test = x_test.sort_index()
    y_train = y_train.sort_index()
    y_test = y_test.sort_index()

    # Crea e addestra il modello di albero di decisione per la regressione
    # decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
    #                              # , class_weight={1: 1.2, 0: 1}
    #                              # , class_weight='balanced'
    #                              , max_depth=3  # , ccp_alpha= 0.00001
    #                              # , max_features= 3
    #                              ).fit(x_train, y_train)
    
    # Crea e addestra il modello di LogRegression
    logRegModel = trainLogRegModel(x_train, y_train, class_weight=classWeight)

    # Crea e addestra il modello di Random Forest
    rfModel = RandomForestClassifier(random_state=42, n_estimators=200, 
                                min_samples_split=80, max_depth=3, class_weight=classWeight).fit(x_train, y_train) 

    # df_test = dataTest[dataTest.index.isin(x_test.index)].copy()
    # df_train = dataTrain[dataTrain.index.isin(x_train.index)].copy()
    df_test = data[data.index.isin(x_test.index)].copy()
    df_train = data[data.index.isin(x_train.index)].copy()
    # print(len(df_test), len(df_train))
    df_train['LogRegPred'] = logRegModel.predict(x_train)
    # df_train['DecTreePred'] = decTreeModel.predict(x_train)
    df_train['RFPred'] = rfModel.predict(x_train)

    df_test['LogRegPred'] = logRegModel.predict(x_test)
    # df_test['DecTreePred'] = decTreeModel.predict(x_test)
    df_test['RFPred'] = rfModel.predict(x_test)
    
    # Ciclo sui modelli
    # models = ['LogReg', 'DecTree', 'RF']
    models = ['LogReg', 'RF']
    rows = []

    for model in models:
        # Metrics
        train_accuracy = round(
            100*accuracy_score(y_train, df_train[model+'Pred']), 2)
        test_accuracy = round(
            100*accuracy_score(y_test, df_test[model+'Pred']), 2)
        
        # test_precision = precision_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, df_test[model+'Pred'])
        print(idx, time.strftime("%H:%M:%S", time.localtime()),test_accuracy, 'Trying with following combo', combo)
        
        # Save Results
        new_row = pd.Series({
            'Model': model,
            'Combo': str(combo),
            'TrainAccuracy': train_accuracy,
            'TestAccuracy': test_accuracy,
            'TestF1': test_f1,
            "UnderTestAccuracy": class_accuracy(y_test, df_test[model+'Pred'])['Under'],
            "OverTestAccuracy": class_accuracy(y_test, df_test[model+'Pred'])['Over']
            # 'BestParams': model.get_params,

        })
        # print(new_row)
        dataframe = pd.concat(
            [dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("../Dataframe/"+str(time_stamp) + "all.xlsx")

# #  print(result)

0 22:12:48 67.38 Trying with following combo ('EloDiff',)
0 22:12:48 65.46 Trying with following combo ('EloDiff',)


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])


1 22:12:52 67.38 Trying with following combo ('PointsDifference',)
1 22:12:52 64.57 Trying with following combo ('PointsDifference',)


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])


2 22:12:58 61.59 Trying with following combo ('HomeGoalsRatio',)
2 22:12:58 57.99 Trying with following combo ('HomeGoalsRatio',)
3 22:13:04 52.22 Trying with following combo ('AwayGoalsRatio',)
3 22:13:04 55.28 Trying with following combo ('AwayGoalsRatio',)
4 22:13:09 58.19 Trying with following combo ('GoalsDifference',)
4 22:13:09 53.48 Trying with following combo ('GoalsDifference',)
5 22:13:12 67.38 Trying with following combo ('ConcededGoalsDifference',)
5 22:13:12 60.34 Trying with following combo ('ConcededGoalsDifference',)


...

752 22:59:02 62.46 Trying with following combo ('AwayDraws', 'feat2')
752 22:59:02 59.34 Trying with following combo ('AwayDraws', 'feat2')
753 22:59:03 66.85 Trying with following combo ('AwayDraws', 'FormDiff')
753 22:59:03 62.54 Trying with following combo ('AwayDraws', 'FormDiff')
754 22:59:05 66.63 Trying with following combo ('AwayDraws', 'RecentFormDiff')
754 22:59:05 65.43 Trying with following combo ('AwayDraws', 'RecentFormDiff')
755 22:59:07 58.8 Trying with following combo ('AwayDraws', 'GoalCumulativeSum')
755 22:59:07 62.81 Trying with following combo ('AwayDraws', 'GoalCumulativeSum')
756 22:59:08 57.63 Trying with following combo ('AwayLosses', 'UltimoScontroDiretto')
756 22:59:08 53.38 Trying with following combo ('AwayLosses', 'UltimoScontroDiretto')
757 22:59:09 56.78 Trying with following combo ('AwayLosses', 'Last3PointsDifference')
757 22:59:09 57.34 Trying with following combo ('AwayLosses', 'Last3PointsDifference')
758 22:59:12 60.19 Trying with following combo 

c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])


768 22:59:33 66.86 Trying with following combo ('AwayLosses', 'RecentFormDiff')
768 22:59:33 62.72 Trying with following combo ('AwayLosses', 'RecentFormDiff')
769 22:59:35 58.72 Trying with following combo ('AwayLosses', 'GoalCumulativeSum')
769 22:59:35 61.25 Trying with following combo ('AwayLosses', 'GoalCumulativeSum')
770 22:59:36 56.6 Trying with following combo ('UltimoScontroDiretto', 'Last3PointsDifference')
770 22:59:36 58.45 Trying with following combo ('UltimoScontroDiretto', 'Last3PointsDifference')
771 22:59:38 59.71 Trying with following combo ('UltimoScontroDiretto', 'GoalRatioDifference')
771 22:59:38 60.4 Trying with following combo ('UltimoScontroDiretto', 'GoalRatioDifference')
772 22:59:41 56.85 Trying with following combo ('UltimoScontroDiretto', 'EwmaGoalsSum')
772 22:59:41 56.68 Trying with following combo ('UltimoScontroDiretto', 'EwmaGoalsSum')
773 22:59:42 58.13 Trying with following combo ('UltimoScontroDiretto', 'GoalsSum')
773 22:59:42 55.56 Trying with f

c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:285: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_0 = cm[0, 0] / (cm[0, 0] + cm[1, 0])


859 23:03:01 56.07 Trying with following combo ('FormDiff', 'GoalCumulativeSum')
859 23:03:01 55.11 Trying with following combo ('FormDiff', 'GoalCumulativeSum')
860 23:03:02 56.1 Trying with following combo ('RecentFormDiff', 'GoalCumulativeSum')
860 23:03:02 52.21 Trying with following combo ('RecentFormDiff', 'GoalCumulativeSum')


In [21]:
# dataframe.to_excel("../Dataframe/"+str(time_stamp) + "all.xlsx")